In [ ]:
import pandas as pd
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.optimizers import RMSprop, Adam
import math
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv('../input/224-landmarks/feature_extra.csv')
emotion_label_to_text = {0:'anger', 1:'disgust', 2:'fear', 3:'happiness', 4: 'sadness', 5: 'surprise', 6: 'neutral'}
print(df.shape)
df.emotion.value_counts()

In [ ]:
"""
  `image` is a 2-D numpy array
  `height` and `width` are the desired spatial dimension of the new 2-D array.
  """
def bilinear_resize(image, height, width):
    img_height, img_width = image.shape[:2]
    resized = np.empty([height, width])
    x_ratio = float(img_width - 1) / (width - 1) if width > 1 else 0
    y_ratio = float(img_height - 1) / (height - 1) if height > 1 else 0

    for i in range(height):
        for j in range(width):
            x_l, y_l = math.floor(x_ratio * j), math.floor(y_ratio * i)
            x_h, y_h = math.ceil(x_ratio * j), math.ceil(y_ratio * i)

            x_weight = (x_ratio * j) - x_l
            y_weight = (y_ratio * i) - y_l

            a = image[y_l, x_l]
            b = image[y_l, x_h]
            c = image[y_h, x_l]
            d = image[y_h, x_h]

            pixel = a * (1 - x_weight) * (1 - y_weight) + b * x_weight * (1 - y_weight) +  c * y_weight * (1 - x_weight) + d * x_weight * y_weight

            resized[i][j] = pixel

    return resized

#### Feature visualization

In [ ]:
i = 1200
img_str = df.loc[i]['pixels']
img_int = [int(x) for x in img_str.split()]
img = np.array(img_int).reshape(48,48)
img = bilinear_resize(img,224,224)

bbx = [int(x)for x in df.loc[i]['bbx'].split()]
img = img[bbx[2]:bbx[3],bbx[0]:bbx[1]]

lm_str = df.loc[i]['relative_lm']
lm_int = [int(x) for x in lm_str.split()]
lm = np.array(lm_int).reshape(68,2)

l_c = np.array([lm[39,0],lm[39,1]])
r_c = np.array([lm[42,0],lm[42,1]])

plt.figure(figsize = (10,10))
for i in range(17,22):
    dis = np.vstack((lm[i,:], l_c))
    plt.plot(dis[:,0],dis[:,1],c ='#E238EC' )
for i in range(22,27):
    dis = np.vstack((lm[i,:], r_c))
    plt.plot(dis[:,0],dis[:,1],c ='#E238EC' )

for j in range(36,42):
    dis = np.vstack((lm[j,:], l_c))
    plt.plot(dis[:,0],dis[:,1],c ='#E0B0FF' )
for j in range(42,48):
    dis = np.vstack((lm[j,:], r_c))
    plt.plot(dis[:,0],dis[:,1],c ='#E0B0FF' )


nose_c = np.array([lm[33,0],lm[33,1]])
for j in range(48,61):
    dis = np.vstack((lm[j,:], nose_c))
    plt.plot(dis[:,0],dis[:,1],c ='#7DFDFE' )

dis = np.vstack((lm[64,:], nose_c))
plt.plot(dis[:,0],dis[:,1],c ='#7DFDFE' )

width = np.vstack((lm[54,:],lm[48,:]))
height = np.vstack((lm[57,:],lm[51,:]))

plt.plot(width[:,0],width[:,1],c ='#7DFDFE' )
plt.plot(height[:,0],height[:,1],c ='#7DFDFE' )

plt.imshow(img,cmap = 'gray')
plt.scatter(lm[:,0],lm[:,1],c = '#1589FF',s  =40)

plt.axis('off')
plt.show()

### Prepare Data

In [ ]:
train = df[df['Usage']=='Training'].copy().reset_index()
test = df[df['Usage']=='PrivateTest'].copy().reset_index()
val = df[df['Usage']=='PublicTest'].copy().reset_index()
# y data
y_train = np.array(train['emotion'])
y_test = np.array(test['emotion'])
y_val = np.array(val['emotion'])

#### Genrate training data X

In [ ]:
def generate_X(df):
    X = np.empty([df.shape[0],176])
    for i in range(df.shape[0]):
        lm = [float(x) for x in df.loc[i]['relative_lm'].split()][:136]

        brow_dis = [float(x) for x in df.loc[i]['brow_distance'].split()]
        
        eye_dis = [float(x) for x in df.loc[i]['eyes_distance'].split()]

        mouth_dis = [float(x) for x in df.loc[i]['mouth_distance'].split()]

        mouth_width = [float(df.loc[i]['mouth_width'])]
        mouth_height = [float(df.loc[i]['mouth_height'])]
        
        mouth_upward = [float(x) for x in df.loc[i]['mouth_upward'].split()]
        
        all_feature = lm+brow_dis+eye_dis+mouth_dis+mouth_width+mouth_height+mouth_upward
        
        X[i,:] = all_feature
    return X

X_train = generate_X(train)
X_test = generate_X(test)
X_val = generate_X(val)

In [ ]:
from sklearn import preprocessing
scaler = preprocessing.StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)
X_val = scaler.transform(X_val)

In [ ]:
print(X_train.shape,X_test.shape,X_val.shape)

#### Get training data y

In [ ]:
print(np.unique(y_train, return_counts=True))
print(np.unique(y_test, return_counts=True))
print(np.unique(y_val, return_counts=True))

In [ ]:
y_train = keras.utils.to_categorical(y_train, 7)
y_test = keras.utils.to_categorical(y_test, 7)
y_val= keras.utils.to_categorical(y_val, 7)

### Model

In [ ]:
model = Sequential()
model.add(Dense(176, activation='relu', input_shape=(176,)))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.6))
model.add(Dense(7, activation='softmax'))

model.summary()

model.compile(loss='categorical_crossentropy',
              optimizer= Adam(lr = 1e-4),                             
              metrics=['accuracy'])

In [ ]:
batch_size = 32
epochs =50
from keras.callbacks import EarlyStopping
early_stopping_monitor = EarlyStopping(monitor='val_loss', patience=4)

history = model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    callbacks=[early_stopping_monitor],
                    verbose=1,             
                    validation_data=(X_val, y_val))

# evaluate model performance
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
# Save model
model.save_weights('landmarks_7c.h5')
model.save('lm_model')

In [ ]:
# Plot training process
accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(len(accuracy))


plt.figure(figsize = (10,4))
plt.subplot(121)
plt.plot(epochs, accuracy,'b',label='Training',linewidth = 2)
plt.plot(epochs, val_accuracy,'b--', label='Validation',linewidth = 2)
plt.xlabel('Epoch', size=10)
plt.ylabel('Accuracy', size=10)
plt.title('Accuracy',size = 12)
plt.legend(fontsize=8)

plt.subplot(122)
plt.plot(epochs, loss,'b', label='Training',linewidth = 2)
plt.plot(epochs, val_loss,'b--',label='Validation',linewidth = 2)
plt.xlabel('Epoch', size=10)
plt.ylabel('Loss', size=10)
plt.title('Loss',size = 12)
plt.legend(fontsize=8)

plt.show()

### Model evaluation

In [ ]:
import seaborn as sns
from sklearn import metrics

y_pred = model.predict(X_test)


Y_test = np.argmax(y_test, axis = 1)
Y_pred = np.argmax(y_pred, axis = 1)


cm1 = metrics.confusion_matrix(Y_test, Y_pred)
cm_normalized = cm1.astype('float') / cm1.sum(axis=1)[:, np.newaxis]
labels = ['anger','disgust','fear','happy','sad','surprise','neutral']
plt.figure(figsize=(9,9))
sns.heatmap(cm_normalized, annot=True, fmt=".2f", linewidths=.5, square=True, cmap='Blues',
            cbar_kws={"shrink": .82},annot_kws={"size":12})
plt.ylabel('True emotion', size=17,rotation = 90)
tick_marks = np.array(range(len(labels))) + 0.5
plt.xticks(tick_marks,labels,ha='center',fontsize = 12)
plt.yticks(tick_marks,labels,va='center',fontsize = 12)
plt.xlabel('Predicted emotion', size=14)
plt.show()

### 3 classes model evaluation

In [ ]:
Y_test_3=  np.where(Y_test== 1,0,Y_test)
Y_test_3 =  np.where(Y_test_3== 2, 0,Y_test_3)
Y_test_3 =  np.where(Y_test_3== 3, 1,Y_test_3)
Y_test_3 =  np.where(Y_test_3== 4, 0,Y_test_3)
Y_test_3 =  np.where(Y_test_3== 5, 1,Y_test_3)
Y_test_3 =  np.where(Y_test_3== 6, 2,Y_test_3)

Y_pred_3 =  np.where(Y_pred== 1,0,Y_pred)
Y_pred_3 =  np.where(Y_pred_3, 0,Y_pred_3)
Y_pred_3 =  np.where(Y_pred== 3, 1,Y_pred_3)
Y_pred_3 =  np.where(Y_pred== 4, 0,Y_pred_3)
Y_pred_3 =  np.where(Y_pred== 5, 1,Y_pred_3)
Y_pred_3 =  np.where(Y_pred== 6, 2,Y_pred_3)

cm1 = metrics.confusion_matrix(Y_test_3, Y_pred_3)
cm_normalized = cm1.astype('float') / cm1.sum(axis=1)[:, np.newaxis]
# labels = ['anger','disgust','fear','happy','sad','surprise','neutral']
labels = ['negative','positive','nutreul']
plt.figure(figsize=(9,9))
sns.heatmap(cm_normalized, annot=True, fmt=".2f", linewidths=.5, square=True, cmap='Blues',
            cbar_kws={"shrink": .82},annot_kws={"size":12})
plt.ylabel('True emotion', size=17,rotation = 90)
tick_marks = np.array(range(len(labels))) + 0.5
plt.xticks(tick_marks,labels,ha='center',fontsize = 12)
plt.yticks(tick_marks,labels,va='center',fontsize = 12)
plt.xlabel('Predicted emotion', size=14)
plt.show()

In [ ]:
import json

with open('training_lm.json', 'w') as f:
    json.dump(history.history, f)

cfm = {}
cfm['y_pred'] = [int(x) for x in Y_pred]
cfm['y_test'] = [int(x) for x in Y_test]
with open('comfusion_matrix_7c.json', 'w') as f:
    json.dump(cfm, f)
    
cfm3 = {}
cfm3['y_pred'] = [int(x) for x in Y_pred_3]
cfm3['y_test'] = [int(x) for x in Y_test_3]
with open('comfusion_matrix_3c.json', 'w') as f:
    json.dump(cfm3, f)